### Code 2 Sub-selection

In [9]:
from netCDF4 import Dataset
import numpy as np
import numpy.ma as ma
import pandas as pd
pd.set_option('max_columns', None)
from scipy.io import loadmat  # this is the SciPy module that loads mat-files
import scipy.io as sio
from itertools import islice
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

#### [0] Load data
zos data along Bathymetry 300 for CMIP6 models and CMEMS observations

In [10]:
#[1] Select section and general parameters
m=loadmat('zos_data_B300_section.mat')
ndata = {n: m['row'][n][0,0] for n in m['row'].dtype.names}
dfm=pd.DataFrame.from_dict(dict((column, ndata[column][0]) for column in [n for n, v in ndata.items() if v.size == 1]))
NSeg=dfm['N'][0];KB_bloom=dfm['KB'][0];model=dfm['model'][0];
ns=dfm['Nstr'][0]; ne=dfm['Nend'][0];ss=dfm['Sstr'][0]; se=dfm['Send'][0];

#[2] KB data
G=int(KB_bloom[-1]) 
for Q in ['2Q']:     
    file='KB_data_{}2014L10G{}.csv'.format(Q,G)
    Kdf=pd.read_csv(file)
    kb=Kdf.iloc[:,-1].copy()
    kb[kb>0]=1
    kb[kb.isnull()]=0
    KBCC=Kdf['max_cells/L_raw_b1e5'].copy()
    KBCC[pd.isna(Kdf['n_days_bloom'])]=0
    if Q=='Q':
        KBQ=kb.to_numpy()
        KBCCQ=KBCC.to_numpy()
    elif Q=='2Q':
        KB2Q=kb.to_numpy()
        KBCC2Q=KBCC.to_numpy()
if KB_bloom[0]=='Q':
    nm=3;Q='Q';KB=KBQ    
elif KB_bloom[0]=='2':
    nm=6;Q='2Q';KB=KB2Q
print(file)
print(KB_bloom,Q,NSeg,model,ne,ns,ss,se)

#[3] observation data
#(Obs)   CMEMS.AVISO-1-0.phy-001-030.r1.Omon.zos.gn (1 realization)
zosO=np.loadtxt('zos_data_B300_10_phy001_030_r1.csv',delimiter=',')
print('zos_obs:',zosO.shape)

#[4] zos model data
#(0-1)   CMIP6.HighResMIP.NCAR.CESM1-CAM5-SE-HR.hist-1950.r1i1p1f1.Omon.zos.gn (1 realization)   [Q 3]
#(1-2)   CMIP6.HighResMIP.CMCC.CMCC-CM2-HR4.hist-1950.r1i1p1f1.Omon.zos.gn (1 realization) [Q 2]
#(2-3)   CMIP6.HighResMIP.CMCC.CMCC-CM2-VHR4.hist-1950.r1i1p1f1.Omon.zos.gn (1 realization) [Q 2]
#(3-6)   CMIP6.HighResMIP.CNRM-CERFACS.CNRM-CM6-1-HR.hist-1950.r1i1p1f2.Omon.zos.gn (3 realizations)  [Q 1]
#(6-7)   CMIP6.CMIP.CNRM-CERFACS.CNRM-CM6-1-HR.historical.r1i1p1f2.Omon.zos.gn (1 realizations) [Q 1]
#(7-12)  CMIP6.CMIP.E3SM-Project.ES3M-1-0.historical.r1i1p1f1.Omon.zos.gr (5 realizations) [Q 0]
#(12-15) CMIP6.HighResMIP.EC-Earth-Consortium.EC-Earth3P-HR.hist-1950.r1i1p2f1.Omon.zos.gn (3 realizations) [Q 0]
#(15-18) CMIP6.HighResMIP.EC-Earth-Consortium.EC-Earth3P.hist-1950.r1i1p2f1.Omon.zos.gn (3 realizations) [Q 4]
#(18-24) CMIP6.HighResMIP.ECMWF.ECMWF-IFS-HR.hist-1950.r1i1p1f1.Omon.zos.gn (6 realizations) [Q 5]
#(24-27) CMIP6.HighResMIP.ECMWF.ECMWF-IFS-MR.hist-1950.r1i1p1f1.Omon.zos.gn (3 realizations)[Q 5]
#(27-28) CMIP6.CMIP.NOAA-GFDL.GFDL-CM4.historical.r1i1p1f1.Omon.zos.gn (1 realizations) [Q 4]
#(28-30) CMIP6.CMIP.NOAA-GFDL.GFDL-ESM4.historical.r2i1p1f1.Omon.zos.gn (2 realizations) [Q 3]
#(30-31) CMIP6.HighResMIP.NERC.HadGEM3-GC31-HH.hist-1950.r1i1p1f1.Omon.zos.gn (1 realization)  [Q 5]
#(31-34) CMIP6.HighResMIP.MOHC.HadGEM3-GC31-HM.hist-1950.r1i1p1f1.Omon.zos.gn (3 realizations) [Q 5]
#(34-37) CMIP6.HighResMIP.MOHC.HadGEM3-GC31-MM.hist-1950.r1i1p1f1.Omon.zos.gn (3 realizations) [Q 5]
#(37-41) CMIP6.CMIP.MOHC.HadGEM3-GC31-MM.historical.r1i1p1f3.Omon.zos.gn (4 realizations) [Q 5]
zosMRaw=np.load('zos_data_B300_543210.npy')
print('zos_model:', zosMRaw.shape)
print ('Number of members:', zosMRaw.shape[0])


#Model info
df=pd.read_csv('zos_data_B300_members_score.csv',index_col=0)
display(df)

KB_data_2Q2014L10G5.csv
2QL10G5 2Q B-300 MSXP 22 9 59 78
zos_obs: (264, 119)
zos_model: (41, 264, 119)
Number of members: 41


,MIP_ERA,Activity,Institution_ID,Source_ID,Experiment_ID,Variant_Label,n_realizations,zos_size,Resolution,nominal_resolution,Score,Member
0,CMIP6,HighResMIP,NCAR,CESM1-CAM5-SE-HR,hist-1950,r1i1p1f1,1,"[3600,2400,780]",HR,25 km,1.0,0.0
1,CMIP6,HighResMIP,CMCC,CMCC-CM2-HR4,hist-1950,r1i1p1f1,1,"[1442,1051,12]",HR,25 km,2.0,1.0
2,CMIP6,HighResMIP,CMCC,CMCC-CM2-VHR4,hist-1950,r1i1p1f1,1,"[1442,1051,12]",HR,25 km,2.0,1.0
3,CMIP6,HighResMIP,CNRM-CERFACS,CNRM-CM6-1-HR,hist-1950,r1i1p1f2,3,"[1442,1050,120]",HR,25 km,1.0,2.0
4,CMIP6,HighResMIP,CNRM-CERFACS,CNRM-CM6-1-HR,hist-1950,r2i1p1f2,0,"[1442,1050,120]",HR,25 km,1.0,2.0
5,CMIP6,HighResMIP,CNRM-CERFACS,CNRM-CM6-1-HR,hist-1950,r3i1p1f2,0,"[1442,1050,120]",HR,25 km,1.0,2.0
6,CMIP6,CMIP,CNRM-CERFACS,CNRM-CM6-1-HR,historical,r1i1p1f2,1,"[1442,1050,1980]",HR,25 km,1.0,2.0
7,CMIP6,CMIP,E3SM-Project,E3SM-1-0,historical,r1i1p1f1,5,"[360,180,60]",LR,100 km,0.0,3.0
8,CMIP6,CMIP,E3SM-Project,E3SM-1-0,historical,r2i1p1f1,0,"[360,180,60]",LR,100 km,0.0,3.0
9,CMIP6,CMIP,E3SM-Project,E3SM-1-0,historical,r3i1p1f1,0,"[360,180,60]",LR,100 km,0.0,3.0


### [1] Sub-selection predictors 
For Loop Current north (LC-N) and Loop Current south (LC-S) given 2Q (i.e., 6 month perid): <br>
(1) resolve observed physical phenomena (Yes / No), (2) frequency of an oscillation(LC-N, LC-S), <br>
(3) temproal-match(LC-N, LC-S,Total), (4) RMSE(Total) for each member, model, and group (Table 1-3)

In [11]:
def predictos(resm,member,KB,LCO,LC,Institution_ID,Source_ID,ensemble_size,Flag):
    
    #Info
    if Flag==1:
        resm.loc[member,'Institution_ID']=Institution_ID
        resm.loc[member,'Source_ID']=Source_ID
    resm.loc[member,'e_size']=ensemble_size
    
    #KB Blooms and LC counts
    resm.loc[member,'KB']=(KB>0).sum()
    resm.loc[member,'LCN']=(LC>=0).sum()
    resm.loc[member,'LCS']=(LC<0).sum()
    resm.loc[member,'LCN_NB']=((LC>=0) & (KB==0)).sum()
    resm.loc[member,'LCN_B']=((LC>=0) & (KB>0)).sum()
    resm.loc[member,'LCS_NB']=((LC<0)  & (KB==0)).sum()
    resm.loc[member,'LCS_B']=((LC<0)  & (KB>0)).sum()
    resm.loc[member,'Err_KB']= np.round(resm.loc[member,'LCS_B']/resm.loc[member,'KB'],decimals=3)

    #Temporal match between observation and model
    resm.loc[member,'Match_LCN']=((LC>=0) & (LCO>=0)).sum()    
    resm.loc[member,'Match_LCS']=((LC<0) & (LCO<0)).sum()  
    resm.loc[member,'Match_Tot']=resm.loc[member,'Match_LCN']+resm.loc[member,'Match_LCS']

    #Temporal error between observation and model
    resm.loc[member,'Err_LCN']=0
    resm.loc[member,'Err_LCS']=0
    resm.loc[member,'Err_Tot']=0

    #Temporal error between AVISO and model
    if member =='obs':
        resm.loc[member,'Err_LCN']=0
        resm.loc[member,'Err_LCS']=0
        resm.loc[member,'Err_Tot']=0
    else:
        resm.loc[member,'Err_LCN']=np.round((resm.loc['obs','LCN']-resm.loc[member,'Match_LCN'])/resm.loc['obs','LCN'],decimals=3)
        resm.loc[member,'Err_LCS']=np.round((resm.loc['obs','LCS']-resm.loc[member,'Match_LCS'])/resm.loc['obs','LCS'],decimals=3)
        resm.loc[member,'Err_Tot']=np.round((len(LCO)-resm.loc[member,'Match_Tot'])/len(LCO),decimals=3)

    #RMSE between AVISO and model
    resm.loc[member,'RMSE']=np.round(np.sqrt(np.mean(np.square(LC-LCO)))*1e2,decimals=2)

    return resm

In [12]:
print('zos data processing steps MSXP: mean_segment(mean_ensemble).delta_north_south.max_period')

#(1) Ensembles 
NME=['3210',   '321X',     '32XX',      '3XXX',       'XXX0']
ME=[[3,2,1,0], [3,2,1,-1], [3,2,-1,-1], [3,-1,-1,-1], [-1,-1,-1,0]]
# NME=['3XXX']
# ME=[[3,-1,-1,-1]]
Disp=0

#(2)Create results dataframe
members=['obs', *[*NME]]
columns=['e_size', 'KB','LCN','LCS','LCN_NB','LCN_B','LCS_NB','LCS_B','Err_KB', \
         'Match_LCN','Match_LCS','Match_Tot','Err_LCN','Err_LCS','Err_Tot','RMSE']
resm = pd.DataFrame(columns = columns,index=members)

#(3) Create zos dataframe
columns=['KB','obs', *[*NME]]
Q=pd.date_range('1993-01-01', periods=44, freq='2Q',closed='left')
dfzos = pd.DataFrame(columns=columns,index=Q)
dfzos.KB=KB

#(4) Observation data processing 
DO=(np.nanmean(zosO[:,ns:ne], axis=1) - np.nanmean(zosO[:,ss:se], axis=1))
LCO=DO.reshape((-1,nm),order='C').max(axis=1)
member='obs'
ensemble_size=1
resm=predictos(resm,'obs',KB,LCO,LCO,Institution_ID='', Source_ID='', ensemble_size='', Flag=2)
dfzos.obs=LCO

for nme,me in zip(NME,ME):    
    
    #[1] Step 1: Collect model runs data 
    #Initalize ensemble: zos data and info
    ZOS=[]
    df_ensemble = df[0:0]
    
    #(1.1)Ensemble data and info
    for index, row in df.iterrows():
        Score=row['Score']
        Institution_ID=row['Institution_ID']
        if Score==me[0] or Score==me[1] or Score==me[2] or Score==me[3]:
            if (Institution_ID != 'CMEMS'):
                temp=zosMRaw[index,:,:]
                temp[temp>1e3]=np.nan
                
                ZOS.append(temp) 
                df_ensemble.loc[index]=row
    ZOS= np.stack(ZOS)
    ZOSN=ZOS[:,:,ns:ne]
    ZOSS=ZOS[:,:,ss:se]
    print('Step 1: Collect zos data {} for north {} and south {} segments for all model runs for multi-model ensemble {}'.\
          format(ZOS.shape,ZOSN.shape,ZOSS.shape,nme))
    
    #(1.2) Save data for optimization
    m['ZOS']=ZOS
    m['ZOSN']=ZOSN
    m['ZOSS']=ZOSS
    m['Member']=df_ensemble.loc[:,['Score','Member']].to_numpy()
    m['row']['Ensemble'][0,0][0]=nme
    df_ensemble.to_csv('zos_data_B300_opt_T{}.csv'.format(nme))

    #[2] Process ensemble (ensemble mean and std)
    #zosA=np.nanmean(ZOS, axis=0)
    #zosAstd=np.nanstd(ZOS,axis=0)
    zosAN=np.nanmean(ZOSN, axis=0)
    zosAS=np.nanmean(ZOSS, axis=0)
      
    #Display ensemble info and zos data size
    if nme=='XXX0':
        print('For each multi-model ensemble:')
        print('Step 2: Average zos data of all model runs for north segment {} and south segment {} '.format(zosAN.shape,zosAS.shape))
    if Disp>0:
        display(df_ensemble)
        print('zos data',nme,':',ZOS.shape,zosAN.shape,zosAS.shape)

    #[3] Data processing MSXP: mean_segment(delta-north-south), max_period
    
    #(3.1) Mean segment
    std=0
    if std==0:
        zosMN=zosAN
        zosMS=zosAS
    elif std==1:
        zosM=zosAstd
    #DMN=np.nanmean(zosM[:,ns:ne], axis=1)
    #DMS=np.nanmean(zosM[:,ss:se], axis=1)
    DMN=np.nanmean(zosMN, axis=1)
    DMS=np.nanmean(zosMS, axis=1)
    if nme=='XXX0':
        print('Step 3: Average zos data of north segment{} and south segment {}'.format(DMN.shape,DMS.shape))
    
    #(3.2) Delta north and south
    DM=DMN-DMS 
    if nme=='XXX0':
        print('Step 4: Subtract zos data of north segment from south segment {}'.format(DM.shape))
    
    #(3.3) Maximum delta zos per period 
    LCM=DM.reshape((-1,nm),order='C').max(axis=1)
    if nme=='XXX0':
        print('Step 5: Select maximum delta zos in the 6-month interval {} given 22-year study period'.format(LCM.shape))
    
    #(3.4) Collect data per model run
    dfzos.loc[:,nme]=LCM
    
    #(3.5) Save data for optimization
    m['LCO']=LCO
    m['LCM']=LCM
    mfile='zos_data_B300_opt_R{}.mat'.format(nme)
    sio.savemat(mfile,m)

    #[4] Calculate predictors 
    ensemble_size=ZOS.shape[0]
    resm=predictos(resm,nme,KB,LCO,LCM,Institution_ID='', Source_ID='', ensemble_size=ensemble_size, Flag=2)
    

#Display results table
resm.iloc[0,0]=1
display(resm)

#Save table
resm.to_csv('res_Table3_Subset_selection.csv')

zos data processing steps MSXP: mean_segment(mean_ensemble).delta_north_south.max_period
Step 1: Collect zos data (41, 264, 119) for north (41, 264, 13) and south (41, 264, 19) segments for all model runs for multi-model ensemble 3210
Step 1: Collect zos data (33, 264, 119) for north (33, 264, 13) and south (33, 264, 19) segments for all model runs for multi-model ensemble 321X
Step 1: Collect zos data (28, 264, 119) for north (28, 264, 13) and south (28, 264, 19) segments for all model runs for multi-model ensemble 32XX
Step 1: Collect zos data (20, 264, 119) for north (20, 264, 13) and south (20, 264, 19) segments for all model runs for multi-model ensemble 3XXX
Step 1: Collect zos data (8, 264, 119) for north (8, 264, 13) and south (8, 264, 19) segments for all model runs for multi-model ensemble XXX0
For each multi-model ensemble:
Step 2: Average zos data of all model runs for north segment (264, 13) and south segment (264, 19) 
Step 3: Average zos data of north segment(264,) and s

,e_size,KB,LCN,LCS,LCN_NB,LCN_B,LCS_NB,LCS_B,Err_KB,Match_LCN,Match_LCS,Match_Tot,Err_LCN,Err_LCS,Err_Tot,RMSE
obs,1,15,32,12,17,15,12,0,0,32,12,44,0,0,0,0
3210,41,15,3,41,2,1,27,14,0.933,2,11,13,0.938,0.083,0.705,5.13
321X,33,15,34,10,22,12,7,3,0.2,25,3,28,0.219,0.75,0.364,3.71
32XX,28,15,23,21,17,6,12,9,0.6,17,6,23,0.469,0.5,0.477,3.92
3XXX,20,15,35,9,22,13,7,2,0.133,28,5,33,0.125,0.583,0.25,3.68
XXX0,8,15,0,44,0,0,29,15,1,0,12,12,1,0,0.727,13.52
